In [11]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from typing import Dict, List, Any

# Additional imports for link security testing
import requests
import re
import ssl
import socket
from urllib.parse import urlparse
import whois
import dns.resolver
import certifi
import datetime

In [12]:
class MobileAppSecurityFramework:
    def __init__(self, config_path: str = 'config.json'):
        """
        Initialize the Mobile App Security Testing Framework

        :param config_path: Path to configuration file
        """
        # Load configuration
        try:
            with open(config_path, 'r') as f:
                self.config = json.load(f)
        except FileNotFoundError:
            self.config = {}

        # Initialize components
        self.vulnerability_model = None
        self.dataset = None
        self.preprocessor = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.X_train_scaled = None
        self.X_test_scaled = None

        # Visualization output directory
        self.output_dir = 'security_visualizations'
        os.makedirs(self.output_dir, exist_ok=True)

        # Add link security tester as an attribute
        self.link_security_tester = LinkSecurityTester()

    def load_dataset(self, dataset_path: str):
        """
        Load and preprocess the mobile app security dataset

        :param dataset_path: Path to the dataset CSV
        """
        # Ensure dataset exists
        if not os.path.exists(dataset_path):
            # Generate dataset if not exists
            from mobile_security_dataset import generate_mobile_app_vulnerability_dataset
            generate_mobile_app_vulnerability_dataset(n_samples=1000)

        # Load dataset
        self.dataset = pd.read_csv(dataset_path)

        # Preprocessing
        self.preprocessor = StandardScaler()

        # Separate features and labels
        X = self.dataset.drop('vulnerability_label', axis=1)
        y = self.dataset['vulnerability_label']

        # Split the data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Scale features
        self.X_train_scaled = self.preprocessor.fit_transform(self.X_train)
        self.X_test_scaled = self.preprocessor.transform(self.X_test)

    def build_ml_model(self):
        """
        Build and compile the vulnerability detection neural network
        """
        # Ensure dataset is loaded
        if self.X_train_scaled is None:
            raise ValueError("Dataset must be loaded first. Call load_dataset() method.")
        self.vulnerability_model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(self.X_train_scaled.shape[1],)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        self.vulnerability_model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

    def train_model(self, epochs: int = 50, batch_size: int = 32):
        """
        Train the vulnerability detection model

        :param epochs: Number of training epochs
        :param batch_size: Training batch size
        """
        # Ensure model is built
        if self.vulnerability_model is None:
            raise ValueError("Model must be built first. Call build_ml_model() method.")

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        history = self.vulnerability_model.fit(
            self.X_train_scaled,
            self.y_train,
            validation_split=0.2,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping]
        )

        # Visualize training history
        self.plot_training_history(history)

        return history

    def plot_training_history(self, history):
        """
        Visualize model training history

        :param history: Model training history
        """
        plt.figure(figsize=(12, 4))

        # Loss plot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        # Accuracy plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, 'training_performance.png'))
        plt.close()

    def evaluate_model(self):
        """
        Evaluate the model's performance

        :return: Comprehensive performance metrics
        """
        # Predict on test data
        y_pred = (self.vulnerability_model.predict(self.X_test_scaled) > 0.5).astype(int)

        # Generate classification report
        report = classification_report(
            self.y_test,
            y_pred,
            target_names=['No Vulnerability', 'Vulnerability']
        )

        # Confusion Matrix
        cm = confusion_matrix(self.y_test, y_pred)

        # Visualize confusion matrix
        self.visualization_confusion_matrix(cm)

        return {
            'classification_report': report,
            'confusion_matrix': cm
        }

    def visualization_confusion_matrix(self, cm):
        """
        Visualize confusion matrix with detailed annotations

        :param cm: Confusion matrix from model evaluation
        """
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['No Vulnerability', 'Vulnerability'],
                    yticklabels=['No Vulnerability', 'Vulnerability'])
        plt.title('Vulnerability Detection Confusion Matrix')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, 'confusion_matrix.png'))
        plt.close()

    def vulnerability_heatmap(self):
        """
        Create a heatmap of vulnerability features correlation
        """
        plt.figure(figsize=(12, 8))

        # Correlation matrix of vulnerability features
        correlation_matrix = self.dataset.drop('vulnerability_label', axis=1).corr()

        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                    square=True, linewidths=0.5, cbar_kws={"shrink": .8})
        plt.title('Mobile App Security Features Correlation Heatmap')
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, 'vulnerability_correlation_heatmap.png'))
        plt.close()

    def detect_vulnerabilities(self, app_features: Dict[str, Any]) -> Dict[str, float]:
        """
        Detect vulnerabilities in a mobile application

        :param app_features: Dictionary of application features
        :return: Vulnerability scores and types
        """
        # Ensure preprocessor exists
        if self.preprocessor is None:
            raise ValueError("Dataset must be loaded first. Call load_dataset() method.")

        # Convert input to scaled features
        features_array = self.preprocessor.transform(
            pd.DataFrame([app_features])
        )

        # Predict vulnerability probability
        vulnerability_prob = self.vulnerability_model.predict(features_array)[0][0]

        vulnerability_types = {
            'insecure_storage': vulnerability_prob * 0.4,
            'weak_encryption': vulnerability_prob * 0.3,
            'api_misuse': vulnerability_prob * 0.3
        }

        # Visualize sensitive data exposure
        self.sensitive_data_exposure_analysis(vulnerability_types)

        return {
            'total_vulnerability_score': vulnerability_prob,
            'vulnerability_breakdown': vulnerability_types
        }

    def sensitive_data_exposure_analysis(self, vulnerability_types):
        """
        Visualize sensitive data exposure risks

        :param vulnerability_types: Dictionary of vulnerability types and scores
        """
        plt.figure(figsize=(10, 6))
        plt.bar(
            [vuln.replace('_', ' ').title() for vuln in vulnerability_types.keys()],
            [score * 100 for score in vulnerability_types.values()]
        )
        plt.title('Sensitive Data Exposure Risk by Vulnerability Type')
        plt.xlabel('Vulnerability Type')
        plt.ylabel('Exposure Risk (%)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, 'sensitive_data_exposure.png'))
        plt.close()

    def threat_classification_dashboard(self, sample_apps):
        """
        Create a dashboard visualizing threat classifications for multiple apps

        :param sample_apps: List of dictionaries with app features
        """
        vulnerability_scores = []
        app_names = []

        for i, app in enumerate(sample_apps, 1):
            results = self.detect_vulnerabilities(app)
            vulnerability_scores.append(results['total_vulnerability_score'] * 100)
            app_names.append(f'App {i}')

        plt.figure(figsize=(12, 6))

        # Threat level color mapping
        colors = ['green' if score < 30 else 'yellow' if score < 60 else 'red' for score in vulnerability_scores]

        plt.bar(app_names, vulnerability_scores, color=colors)
        plt.title('Threat Classification Dashboard')
        plt.xlabel('Mobile Applications')
        plt.ylabel('Vulnerability Score (%)')
        plt.axhline(y=30, color='green', linestyle='--', label='Low Risk')
        plt.axhline(y=60, color='yellow', linestyle='--', label='Medium Risk')
        plt.axhline(y=100, color='red', linestyle='--', label='High Risk')
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, 'threat_classification_dashboard.png'))
        plt.close()

    def generate_security_report(self, results: Dict[str, Any]) -> str:
        """
        Generate a detailed security report

        :param results: Vulnerability detection results
        :return: Formatted security report
        """
        report = "Mobile App Security Assessment Report\n"
        report += "=" * 40 + "\n\n"

        report += f"Total Vulnerability Score: {results['total_vulnerability_score']:.2%}\n\n"

        report += "Vulnerability Breakdown:\n"
        for vuln_type, score in results['vulnerability_breakdown'].items():
            report += f"- {vuln_type.replace('_', ' ').title()}: {score:.2%}\n"

        return report

    def test_links(self, links: List[str]) -> List[Dict[str, Any]]:
        """
        Test multiple links for security risks

        :param links: List of links to test
        :return: List of link security assessments
        """
        return [self.link_security_tester.validate_url(link) for link in links]

    def generate_link_security_report(self, link_results: List[Dict[str, Any]]) -> str:
        """
        Generate a detailed report for link security tests

        :param link_results: List of link security assessment results
        :return: Formatted security report
        """
        report = "Link Security Assessment Report\n"
        report += "=" * 40 + "\n\n"

        for result in link_results:
            report += f"URL: {result['original_url']}\n"
            report += f"Security Score: {result['security_score']:.2f}/100\n"
            report += f"Overall Status: {'SAFE' if result['is_valid'] else 'RISKY'}\n"

            if result.get('risks'):
                report += "Detected Risks:\n"
                for risk in result['risks']:
                    report += f"- {risk}\n"

            # SSL Information
            if 'ssl_info' in result and result['ssl_info'].get('is_valid'):
                ssl_info = result['ssl_info']
                report += "\nSSL Certificate Details:\n"
                report += f"- Issuer: {ssl_info.get('issuer', 'N/A')}\n"
                report += f"- Expiration: {ssl_info.get('expiration_date', 'N/A')}\n"
                report += f"- Days to Expiry: {ssl_info.get('days_to_expiry', 'N/A')}\n"

            # DNS Information
            if 'dns_info' in result:
                dns_info = result['dns_info']
                report += "\nDNS Details:\n"
                report += f"- A Records: {dns_info.get('a_records', 'N/A')}\n"
                report += f"- Total Records: {dns_info.get('total_records', 'N/A')}\n"

            report += "\n" + "-"*40 + "\n\n"

        return report

In [13]:
def generate_mobile_app_vulnerability_dataset(n_samples=1000):
    """
    Generate a synthetic mobile app vulnerability dataset

    :param n_samples: Number of sample applications
    :return: DataFrame with mobile app security features
    """
    import numpy as np
    import pandas as pd
    from sklearn.datasets import make_classification

    # Generate feature matrix
    X, y = make_classification(
        n_samples=n_samples,
        n_features=10,  # Multiple security-related features
        n_informative=7,
        n_redundant=3,
        random_state=42
    )

    # Create feature names representing security attributes
    feature_names = [
        'storage_encryption_level',
        'api_security_score',
        'data_transmission_security',
        'authentication_strength',
        'input_validation_score',
        'network_communication_security',
        'third_party_library_risk',
        'runtime_permissions_management',
        'code_obfuscation_level',
        'certificate_pinning_implementation'
    ]
    # Create DataFrame
    df = pd.DataFrame(X, columns=feature_names)
    # Add vulnerability label
    df['vulnerability_label'] = y

    # Add some realistic variations
    df['storage_encryption_level'] = np.clip(df['storage_encryption_level'], 0, 1)
    df['api_security_score'] = np.abs(df['api_security_score'])

    # Save dataset
    df.to_csv('mobile_app_vulnerabilities.csv', index=False)
    return df

In [14]:
class LinkSecurityTester:
    def __init__(self):
        """
        Initialize Link Security Tester
        """
        self.bad_link_patterns = [
            r'(https?://)?([^/]+)?(\.)?((bit\.ly)|(tinyurl)|(t\.co))',  # URL shorteners
            r'(https?://)?([^/]+)?(\.)?((exe)|(msi)|(bat)|(cmd)|(scr))$',  # Suspicious file extensions
        ]

    def validate_url(self, url: str) -> Dict[str, Any]:
        """
        Comprehensive URL security validation

        :param url: URL to test
        :return: Detailed security assessment of the link
        """
        # Basic validation
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url

        try:
            # Parse URL
            parsed_url = urlparse(url)

            # Initial checks
            security_report = {
                'original_url': url,
                'is_valid': False,
                'security_score': 0,
                'risks': []
            }

            # Check for suspicious patterns
            if any(re.search(pattern, url, re.IGNORECASE) for pattern in self.bad_link_patterns):
                security_report['risks'].append('Suspicious URL pattern detected')
                security_report['security_score'] -= 20

            # DNS resolution check
            try:
                dns_records = self._check_dns_records(parsed_url.netloc)
                security_report['dns_info'] = dns_records
            except Exception as e:
                security_report['risks'].append(f'DNS resolution failed: {str(e)}')
                security_report['security_score'] -= 15

            # SSL/TLS certificate check
            try:
                ssl_info = self._check_ssl_certificate(parsed_url.netloc)
                security_report['ssl_info'] = ssl_info

                # Adjust security score based on SSL validity
                if ssl_info['is_valid']:
                    security_report['security_score'] += 30
                else:
                    security_report['risks'].append('Invalid or expired SSL certificate')
                    security_report['security_score'] -= 25
            except Exception as e:
                security_report['risks'].append(f'SSL check failed: {str(e)}')
                security_report['security_score'] -= 20

            # Whois information
            try:
                whois_info = self._check_domain_whois(parsed_url.netloc)
                security_report['whois_info'] = whois_info
            except Exception as e:
                security_report['risks'].append(f'Whois lookup failed: {str(e)}')

            # HTTP/HTTPS request check
            try:
                request_result = self._perform_request(url)
                security_report.update(request_result)

                # Adjust security score based on request results
                if request_result['status_code'] == 200:
                    security_report['security_score'] += 20
                else:
                    security_report['risks'].append(f'Suspicious HTTP status: {request_result["status_code"]}')
                    security_report['security_score'] -= 15
            except Exception as e:
                security_report['risks'].append(f'Request failed: {str(e)}')
                security_report['security_score'] -= 25

            # Final security score normalization
            security_report['security_score'] = max(0, min(100, security_report['security_score']))
            security_report['is_valid'] = security_report['security_score'] > 50

            return security_report

        except Exception as e:
            return {
                'original_url': url,
                'is_valid': False,
                'security_score': 0,
                'risks': [f'Validation error: {str(e)}']
            }

    def _check_dns_records(self, domain: str) -> Dict[str, Any]:
        """
        Check DNS records for the domain

        :param domain: Domain to check
        :return: DNS record information
        """
        try:
            # Remove port if present
            domain = domain.split(':')[0]

            # Check A records
            a_records = [str(rdata) for rdata in dns.resolver.resolve(domain, 'A')]

            # Check MX records
            mx_records = [str(rdata.exchange) for rdata in dns.resolver.resolve(domain, 'MX')]

            return {
                'a_records': a_records,
                'mx_records': mx_records,
                'total_records': len(a_records) + len(mx_records)
            }
        except Exception as e:
            raise ValueError(f"DNS resolution error: {str(e)}")

    def _check_ssl_certificate(self, domain: str) -> Dict[str, Any]:
        """
        Check SSL/TLS certificate details

        :param domain: Domain to check certificate
        :return: SSL certificate information
        """
        try:
            # Remove port if present
            domain = domain.split(':')[0]

            # Perform SSL context creation and verification
            context = ssl.create_default_context(cafile=certifi.where())
            with socket.create_connection((domain, 443)) as sock:
                with context.wrap_socket(sock, server_hostname=domain) as secure_sock:
                    cert = secure_sock.getpeercert()

                    # Extract certificate details
                    return {
                        'is_valid': True,
                        'issuer': dict(x[0] for x in cert['issuer']),
                        'subject': dict(x[0] for x in cert['subject']),
                        'expiration_date': datetime.datetime.strptime(
                            cert['notAfter'],
                            '%b %d %H:%M:%S %Y %Z'
                        ),
                        'days_to_expiry': (
                            datetime.datetime.strptime(cert['notAfter'], '%b %d %H:%M:%S %Y %Z')
                            - datetime.datetime.now()
                        ).days
                    }
        except ssl.SSLCertVerificationError:
            return {
                'is_valid': False,
                'error': 'Certificate verification failed'
            }
        except Exception as e:
            raise ValueError(f"SSL certificate check failed: {str(e)}")

    def _check_domain_whois(self, domain: str) -> Dict[str, Any]:
        """
        Retrieve WHOIS information for the domain

        :param domain: Domain to check
        :return: WHOIS information
        """
        try:
            # Remove port if present
            domain = domain.split(':')[0]

            # Perform WHOIS lookup
            w = whois.whois(domain)

            return {
                'domain_name': w.domain_name,
                'registrar': w.registrar,
                'creation_date': w.creation_date[0] if isinstance(w.creation_date, list) else w.creation_date,
                'expiration_date': w.expiration_date[0] if isinstance(w.expiration_date, list) else w.expiration_date,
                'age_in_days': (datetime.datetime.now() -
                    (w.creation_date[0] if isinstance(w.creation_date, list) else w.creation_date)).days
            }
        except Exception as e:
            raise ValueError(f"WHOIS lookup failed: {str(e)}")

    def _perform_request(self, url: str) -> Dict[str, Any]:
        """
        Perform HTTP/HTTPS request to validate link

        :param url: URL to request
        :return: Request details
        """
        try:
            # Perform GET request with timeout and user agent
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10, allow_redirects=True)

            return {
                'status_code': response.status_code,
                'content_type': response.headers.get('Content-Type', ''),
                'redirects': len(response.history),
                'final_url': response.url
            }
        except requests.exceptions.RequestException as e:
            raise ValueError(f"Request failed: {str(e)}")

In [15]:
def main():
       # Ensure dataset is generated
    generate_mobile_app_vulnerability_dataset()

    # Initialize framework
    security_framework = MobileAppSecurityFramework()

    # Load dataset
    security_framework.load_dataset('mobile_app_vulnerabilities.csv')

    # Build ML model
    security_framework.build_ml_model()

    # Train model
    training_history = security_framework.train_model()

    # Evaluate model
    model_performance = security_framework.evaluate_model()
    print("Model Performance:")
    print(model_performance['classification_report'])
    # Vulnerability heatmap
    security_framework.vulnerability_heatmap()

    # Example app vulnerability detection
    sample_apps = [
        {
            'storage_encryption_level': 0.3,
            'api_security_score': 0.2,
            'data_transmission_security': 0.4,
            'authentication_strength': 0.1,
            'input_validation_score': 0.2,
            'network_communication_security': 0.3,
            'third_party_library_risk': 0.4,
            'runtime_permissions_management': 0.2,
            'code_obfuscation_level': 0.1,
            'certificate_pinning_implementation': 0.2
        },
        {
            'storage_encryption_level': 0.7,
            'api_security_score': 0.8,
            'data_transmission_security': 0.9,
            'authentication_strength': 0.6,
            'input_validation_score': 0.7,
            'network_communication_security': 0.8,
            'third_party_library_risk': 0.2,
            'runtime_permissions_management': 0.9,
            'code_obfuscation_level': 0.8,
            'certificate_pinning_implementation': 0.7
        }
    ]
    # Threat classification dashboard
    security_framework.threat_classification_dashboard(sample_apps)

    # Detect vulnerabilities for sample apps
    for app in sample_apps:
        vulnerability_results = security_framework.detect_vulnerabilities(app)
        security_report = security_framework.generate_security_report(vulnerability_results)
        print("\nSecurity Report:")
        print(security_report)

    # Example of link security testing
    links_to_test = [
        'https://www.example.com',
        'https://github.com',
        'http://suspicious-site.com'
    ]
    # Perform link security tests
    link_security_results = security_framework.test_links(links_to_test)

    # Generate and print link security report
    link_security_report = security_framework.generate_link_security_report(link_security_results)
    print("\nLink Security Report:")
    print(link_security_report)

In [16]:

if __name__ == "__main__":
    main()

D:\Projects\Master\APISecurityTester\.venv1\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5364 - loss: 0.6608 - val_accuracy: 0.7125 - val_loss: 0.6049
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6797 - loss: 0.5980 - val_accuracy: 0.7750 - val_loss: 0.5338
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7624 - loss: 0.5336 - val_accuracy: 0.8062 - val_loss: 0.4681
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8156 - loss: 0.4496 - val_accuracy: 0.8438 - val_loss: 0.4035
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8052 - loss: 0.4208 - val_accuracy: 0.8562 - val_loss: 0.3675
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8376 - loss: 0.3738 - val_accuracy: 0.8500 - val_loss: 0.3405
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8470 - loss: 0.3471 - val_accuracy: 0.8687 - val_loss: 0.3116
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8194 - loss: 0.3857 - val_accuracy: 0.8875 - val_loss

In [18]:
# Initialize the security framework
security_framework = MobileAppSecurityFramework()

# Test links
links_to_test = [
    'https://www.example.com',
    'https://github.com',
    'http://suspicious-site.com'
]

# Perform link security tests
link_security_results = security_framework.test_links(links_to_test)

# Generate and print link security report
link_security_report = security_framework.generate_link_security_report(link_security_results)
print(link_security_report)

Link Security Assessment Report

URL: https://www.example.com
Security Score: 35.00/100
Overall Status: RISKY
Detected Risks:
- DNS resolution failed: DNS resolution error: The DNS response does not contain an answer to the question: www.example.com. IN MX

SSL Certificate Details:
- Issuer: {'countryName': 'US', 'organizationName': 'DigiCert Inc', 'commonName': 'DigiCert Global G2 TLS RSA SHA256 2020 CA1'}
- Expiration: 2025-03-01 23:59:59
- Days to Expiry: 76

----------------------------------------

URL: https://github.com
Security Score: 50.00/100
Overall Status: RISKY

SSL Certificate Details:
- Issuer: {'countryName': 'GB', 'stateOrProvinceName': 'Greater Manchester', 'localityName': 'Salford', 'organizationName': 'Sectigo Limited', 'commonName': 'Sectigo ECC Domain Validation Secure Server CA'}
- Expiration: 2025-03-07 23:59:59
- Days to Expiry: 82

DNS Details:
- A Records: ['20.205.243.166']
- Total Records: 6

----------------------------------------

URL: http://suspicious-